In [1]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/phd/datasets/gtex/10/";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/R_stuff"

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [2]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [31]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "Name"))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [32]:
#log1 <- function(x){
#    log(x+1)
#}
#datExpr <- as.data.frame(apply(datExpr,2, log2))
#dimnames(datExpr)[[1]] <- samples

In [33]:
datExpr[1:10,1:5]

,ENSG00000268903.1,ENSG00000269981.1,ENSG00000250575.1,ENSG00000225972.1,ENSG00000229344.1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-1GZ2Q-0011-R6a-SM-9OSWM,1.5407307,1.0756848,0.04130508,3.384729,4.648230
GTEX-RU72-0011-R5A-SM-2TF6U,0.5387131,0.9234649,0.00000000,4.357606,4.573163
GTEX-WHSE-0011-R3A-SM-3P5ZM,1.6813864,1.8539515,0.00000000,2.824944,3.551053
GTEX-14PQA-0011-R1b-SM-6EU1G,1.0888986,1.4068320,0.00000000,2.663053,3.461037
GTEX-13CF2-0011-R1a-SM-5LZY1,0.6294616,0.3673478,0.00000000,2.986187,3.395850
GTEX-1B8L1-0011-R10b-SM-7P8Q4,1.7003749,1.4544202,0.01863528,3.180135,3.350255
GTEX-1445S-3026-SM-5O9BR,1.7047481,1.0739780,0.00000000,2.843746,3.433665
GTEX-1B8SF-0011-R8b-SM-7P8SX,0.4893772,0.5572702,0.24513985,5.333202,5.566052
GTEX-15ER7-0011-R7a-SM-6M46N,0.3870975,0.2946825,0.00000000,2.408745,3.159126


In [34]:
head(allTraits)

,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,⋯,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>
1,GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,⋯,14648800,11999300,0.00315785,14669500,50.0354,0.00310538,0.994740,NA,0,50.1944
2,GTEX-1117F-0426-SM-5EGHI,0,B1,"2 pieces, !5% fibrous connective tissue, delineated (rep)",7.1,Muscle,Muscle - Skeletal,0011907,1220,1119,⋯,13191500,11550200,0.00396788,13340500,50.2809,0.00699464,0.995041,NA,0,49.9455
3,GTEX-1117F-0526-SM-5EGHJ,0,B1,"2 pieces, clean, Monckebeg medial sclerosis, rep delineated",8.0,Blood Vessel,Artery - Tibial,0007610,1221,1120,⋯,13175400,11000300,0.00427126,13150900,49.9535,0.00286826,0.994001,NA,0,50.2667
4,GTEX-1117F-0626-SM-5N9CS,1,B1,"2 pieces, up to 4mm aderent fat/nerve/vessel, delineated",6.9,Blood Vessel,Artery - Coronary,0001621,1243,1098,⋯,15164200,11617900,0.00336188,15291900,50.2096,0.00533653,0.992257,NA,0,50.0865
5,GTEX-1117F-0726-SM-5GIEN,1,B1,"2 pieces, no abnormalities",6.3,Heart,Heart - Atrial Appendage,0006631,1244,1097,⋯,13627100,9266070,0.00344234,13756700,50.2367,0.03058410,0.995711,NA,0,49.9563
6,GTEX-1117F-1326-SM-5EGHH,1,B1,"2 pieces, diffuse mesothelial hyperplasia; ~10% vessel/fibrous tissue (delineated)",5.9,Adipose Tissue,Adipose - Visceral (Omentum),0010414,1277,1066,⋯,13379400,9631060,0.00485657,13408700,50.0547,0.01033100,0.990378,NA,0,50.1311


In [36]:
#labels <- c('primary_site', 'malignacy', 'disease_type', 'stage')
#labels <- c('cases.0.project.primary_site', 'cases.0.project.disease_type')
labels <- c('SMTS', 'SMTSD')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$SAMPID
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-1GZ2Q-0011-R6a-SM-9OSWM,Brain,Brain - Nucleus accumbens (basal ganglia)
GTEX-RU72-0011-R5A-SM-2TF6U,Brain,Brain - Caudate (basal ganglia)
GTEX-WHSE-0011-R3A-SM-3P5ZM,Brain,Brain - Anterior cingulate cortex (BA24)
GTEX-14PQA-0011-R1b-SM-6EU1G,Brain,Brain - Hippocampus
GTEX-13CF2-0011-R1a-SM-5LZY1,Brain,Brain - Hippocampus
GTEX-1B8L1-0011-R10b-SM-7P8Q4,Brain,Brain - Frontal Cortex (BA9)


In [37]:
table( dimnames(datExpr)[[1]]==datTraits$file_name)

< table of extent 0 >

In [38]:
y = datTraits$primary_site

In [39]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [17]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [18]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [26]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 83, minSize = 10)
table(clust)

clust
  1   2 
919  81 

In [40]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [41]:
head(datExpr)

,ENSG00000268903.1,ENSG00000269981.1,ENSG00000250575.1,ENSG00000225972.1,ENSG00000229344.1,ENSG00000198744.5,ENSG00000205116.3,ENSG00000178821.12,ENSG00000233542.1,ENSG00000187730.8,⋯,ENSG00000210117.1,ENSG00000210127.1,ENSG00000210140.1,ENSG00000210144.1,ENSG00000210154.1,ENSG00000210156.1,ENSG00000210176.1,ENSG00000210191.1,ENSG00000210195.2,ENSG00000210196.2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GTEX-1GZ2Q-0011-R6a-SM-9OSWM,1.5407307,1.0756848,0.04130508,3.384729,4.648230,3.467297,1.807797,1.0024684,0.68783309,3.880326,⋯,1.0719260,0.9218751,1.438650,2.286659,2.1161350,1.284262,2.056045,1.7549226,1.517761,0.9309823
GTEX-RU72-0011-R5A-SM-2TF6U,0.5387131,0.9234649,0.00000000,4.357606,4.573163,4.576153,2.289804,0.8763017,0.49127576,3.944490,⋯,3.2414198,1.2068686,1.787584,1.917511,2.5564518,2.466403,5.306781,2.9156062,1.774783,1.7500690
GTEX-WHSE-0011-R3A-SM-3P5ZM,1.6813864,1.8539515,0.00000000,2.824944,3.551053,3.895690,2.185377,1.3974816,1.14708494,4.002229,⋯,0.5388298,0.3014370,1.053266,1.175882,0.5388298,1.005766,1.338941,0.7060634,1.043452,0.7280316
GTEX-14PQA-0011-R1b-SM-6EU1G,1.0888986,1.4068320,0.00000000,2.663053,3.461037,3.312730,3.896300,0.7738051,0.26136376,2.066989,⋯,1.3872939,1.1362292,1.854265,2.155476,1.9470524,1.859262,1.934560,4.1762317,1.295549,1.2739652
GTEX-13CF2-0011-R1a-SM-5LZY1,0.6294616,0.3673478,0.00000000,2.986187,3.395850,4.100989,2.573375,0.3361150,0.06252399,1.214319,⋯,1.3299885,2.5556757,2.679651,3.553632,1.8812954,1.924687,2.402430,1.6084374,3.794590,2.2346274
GTEX-1B8L1-0011-R10b-SM-7P8Q4,1.7003749,1.4544202,0.01863528,3.180135,3.350255,3.110399,1.941328,0.9738046,0.97757396,4.969119,⋯,1.9207389,0.8548408,0.730443,1.352738,1.5623463,1.896119,1.550749,0.8385454,1.244731,2.0163686


In [42]:
rownames(allTraits) <- allTraits$SAMPID
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,SMTS,SMTSD
,<chr>,<chr>
GTEX-1GZ2Q-0011-R6a-SM-9OSWM,Brain,Brain - Nucleus accumbens (basal ganglia)
NA,NA,NA
GTEX-WHSE-0011-R3A-SM-3P5ZM,Brain,Brain - Anterior cingulate cortex (BA24)
GTEX-14PQA-0011-R1b-SM-6EU1G,Brain,Brain - Hippocampus
GTEX-13CF2-0011-R1a-SM-5LZY1,Brain,Brain - Hippocampus
GTEX-1B8L1-0011-R10b-SM-7P8Q4,Brain,Brain - Frontal Cortex (BA9)


In [43]:
collectGarbage();

In [44]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”
Warning message in ends - starts:
“longer object length is not a multiple of shorter object length”


png 
  2

## 2

In [45]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=20, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3000.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3000 of 3000


Warning message:
“executing %dopar% sequentially: no parallel backend registered”
Warning message in eval(xpr, envir = envir):
“Some correlations are NA in block 1 : 3000 .”


   Power SFT.R.sq  slope truncated.R.sq mean.k. median.k. max.k.
1      1    0.153  0.546          0.750  351.00   342.000  677.0
2      2    0.232 -0.379          0.577  121.00   122.000  287.0
3      3    0.465 -0.671          0.606   66.60    62.400  187.0
4      4    0.636 -0.682          0.676   44.60    36.600  139.0
5      5    0.722 -0.820          0.809   32.70    23.500  120.0
6      6    0.723 -0.989          0.858   25.40    16.000  110.0
7      7    0.750 -1.090          0.898   20.40    11.100  101.0
8      8    0.771 -1.160          0.925   16.80     8.130   93.4
9      9    0.801 -1.200          0.944   14.10     5.990   86.8
10    10    0.823 -1.240          0.954   12.00     4.520   80.9
11    12    0.810 -1.310          0.941    9.08     2.540   70.9
12    14    0.805 -1.360          0.933    7.09     1.590   62.7
13    16    0.801 -1.370          0.925    5.69     0.998   55.8
14    18    0.811 -1.360          0.923    4.65     0.646   49.9
15    20    0.816 -1.350 

png 
  2

In [46]:
net = blockwiseModules(datExpr, power = 6,
TOMType = "unsigned", minModuleSize = 30,
reassignThreshold = 0, mergeCutHeight = 0.15,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
  ..Excluding 1 genes from the calculation due to too many missing samples or zero variance.
    ..step 2
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will not use multithreading.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
     ..removing 19 genes from module 1 because their KME is too low.
     ..removing 15 genes from module 2 because their KME is too low.
     ..removing 5 genes from module 3 because their KME is too low.
     ..removing 10 genes from module 4 because their KME is too low.
     ..removing 4 genes from module 5 because their KME is too low.
     ..removin

In [47]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [48]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [ ]:
datTraits$cases.0.project.primary_site <- match(datTraits$cases.0.project.primary_site, unique(datTraits$cases.0.project.primary_site))
datTraits$cases.0.project.disease_type <- match(datTraits$cases.0.project.disease_type, unique(datTraits$cases.0.project.disease_type))

In [49]:
datTraits$SMTS <- match(datTraits$SMTS, unique(datTraits$SMTS))
datTraits$SMTSD <- match(datTraits$SMTSD, unique(datTraits$SMTSD))

In [ ]:
datTraits$primary_site <- match(datTraits$primary_site, unique(datTraits$primary_site))
#datTraits$secondary_site <- match(datTraits$secondary_site, unique(datTraits$secondary_site))
datTraits$malignacy <- match(datTraits$malignacy, unique(datTraits$malignacy))
datTraits$disease_type <- match(datTraits$disease_type, unique(datTraits$disease_type))
datTraits$stage <- match(datTraits$stage, unique(datTraits$stage))

In [50]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [51]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [52]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(datTraits$SMTS);
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

In [53]:
module = "yellow"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

png 
  2

In [74]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

In [75]:
# Read in the probe annotation
annot = read.csv(file = "geneInfo.csv");
# Match probes in the data set to the probe IDs in the annotation file
probes = names(datExpr)
allLLIDs = probes
# $ Choose interesting modules
intModules = c("greenyellow", "darkorange", "darkgreen")
for (module in intModules)
{
# Select module probes
modGenes = (moduleColors==module)
# Get their entrez ID codes
modLLIDs = probes[modGenes];
# Write them into a file
fileName = paste("LocusLinkIDs-", module, ".txt", sep="");
write.table(as.data.frame(modLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)
}
# As background in the enrichment analysis, we will use all probes in the analysis.
fileName = paste("LocusLinkIDs-all.txt", sep="");
write.table(as.data.frame(allLLIDs), file = fileName,
row.names = FALSE, col.names = FALSE, quote=FALSE)

In [ ]:
require("biomaRt")
mart <- useMart("ENSEMBL_MART_ENSEMBL")
mart <- useDataset("hsapiens_gene_ensembl", mart)

ens <- allLLIDs
ensLookup <- gsub("\\.[0-9]*$", "", ens)

annotLookup <- getBM(
  mart=mart,
  attributes=c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"),
  filter="ensembl_gene_id",
  values=ensLookup,
  uniqueRows=TRUE)

annotLookup <- data.frame(
  ens[match(annotLookup$ensembl_gene_id, ensLookup)],
  annotLookup)

colnames(annotLookup) <- c(
  "original_id",
  c("ensembl_transcript_id", "ensembl_gene_id", "gene_biotype", "external_gene_name"))

annotLookup

## 5 

In [56]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
svg("nethier.svg", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [57]:
rownames(allTraits)=allTraits$SAMPID
unique(allTraits[rownames(datTraits),'SMTS'])

[1] "Brain"          NA               "Skin"           "Esophagus"     
 [5] "Blood Vessel"   "Adipose Tissue" "Blood"          "Heart"         
 [9] "Muscle"         "Colon"          "Thyroid"

In [55]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'SMTS']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

Warning message in par(fig = c(0, 1, 0, 1), oma = c(0, 0, 0, 0), mar = c(0, 0, 0, :
“calling par(new=TRUE) with no plot”


png 
  2

In [79]:
collectGarbage()

In [80]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.4, minSize = 10)
table(clust)

clust
   1 
1000 

In [81]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

png 
  2

In [34]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

TOM calculation: adjacency..
..will not use multithreading.
 Fraction of slow calculations: 0.000000
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


ERROR: Error in x[, iy]: subscript out of bounds


In [82]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

ERROR: Error in hclust(as.dist(selectTOM), method = "average"): NaN dissimilarity value.


In [83]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

png 
  2

In [84]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

png 
  2

In [64]:
# 2 14 26 38 50
write.csv(cutree(sampleTree3, k=50), "wgcna/wgcna_level_0_labels.csv")

In [65]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [66]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")